# SQL Assignment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [ ]:
# Note that this is not the same db we have used in course videos, please download from this link
# https://drive.google.com/file/d/1O-1-L1DdNxEK6O6nG2jS31MbrMh-OnXM/view?usp=sharing

In [ ]:
conn = sqlite3.connect("/content/Db-IMDB-Assignment.db")

#### Overview of all tables

In [ ]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [ ]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [ ]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results)
    assert (q1_results.shape == (232,3))

query1 = """
SELECT
DISTINCT
TRIM(Person.Name) Director_Name,
TRIM(Movie.title) Movie_Name,
CAST(SUBSTR(Movie.year,-4) AS INTEGER) year
FROM Movie
JOIN M_Director
ON Movie.MID = M_Director.MID
JOIN Person
ON Person.PID = M_Director.PID
JOIN
(
  SELECT
  MID,GID
  FROM M_Genre
  WHERE
  GID IN (
    SELECT
    GID
    FROM Genre
    WHERE
    Name LIKE '%Comedy%'
  )
)
AS G
ON G.MID = Movie.MID
WHERE
  (CAST(SUBSTR(Movie.year,-4) AS INTEGER) % 400=0)
  OR (CAST(SUBSTR(Movie.year,-4) AS INTEGER) % 4=0)
  AND NOT CAST(SUBSTR(Movie.year,-4) AS INTEGER) % 100=0
"""
grader_1(query1)

             Director_Name                         Movie_Name  year
0             Milap Zaveri                         Mastizaade  2016
1             Danny Leiner  Harold & Kumar Go to White Castle  2004
2           Anurag Kashyap                 Gangs of Wasseypur  2012
3             Frank Coraci        Around the World in 80 Days  2004
4            Griffin Dunne             The Accidental Husband  2008
..                     ...                                ...   ...
227  Siddharth Anand Kumar                        Let's Enjoy  2004
228        Amma Rajasekhar                            Sathyam  2008
229          Oliver Paulus                      Tandoori Love  2008
230            Raja Chanda                        Le Halua Le  2012
231       K.S. Prakash Rao                  Raja Aur Rangeeli  1996

[232 rows x 3 columns]
CPU times: user 77.6 ms, sys: 992 µs, total: 78.6 ms
Wall time: 84.8 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [ ]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))


query2 = """
SELECT
DISTINCT
TRIM(Person.name) Actors_who_played_in_Anand_movie
FROM
Movie
JOIN M_Cast
ON Movie.MID = M_Cast.MID
JOIN Person
ON TRIM(M_Cast.PID) = TRIM(Person.PID)
WHERE
Movie.title = 'Anand'
"""
grader_2(query2)

  Actors_who_played_in_Anand_movie
0                 Amitabh Bachchan
1                    Rajesh Khanna
2                    Sumita Sanyal
3                       Ramesh Deo
4                        Seema Deo
5                   Asit Kumar Sen
6                       Dev Kishan
7                     Atam Prakash
8                    Lalita Kumari
9                           Savita
CPU times: user 310 ms, sys: 6.02 ms, total: 316 ms
Wall time: 316 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [ ]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 ="""
Select p.PID from Person p
inner join
(
    select trim(mc.PID) PD, mc.MID from M_cast mc
where mc.MID
in
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)<1970
)
) r1
on r1.PD=p.PID
"""
query_more_1990 ="""
Select p.PID from Person p
inner join
(
    select trim(mc.PID) PD, mc.MID from M_cast mc
where mc.MID
in
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)>1990
)
) r1
on r1.PD=p.PID """
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question

(4942, 1)
(62570, 1)
True
CPU times: user 334 ms, sys: 9 ms, total: 343 ms
Wall time: 356 ms


In [ ]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """
WITH
query_less_1970 AS
(
  SELECT
  DISTINCT
  TRIM(M_Cast.PID) PID
  FROM
  Movie
  JOIN M_Cast
  ON Movie.MID = M_Cast.MID
  WHERE
  CAST(SUBSTR(Movie.year,-4) AS INTEGER) < 1970
),
query_more_1990 AS
(
  SELECT
  DISTINCT
  TRIM(M_Cast.PID) PID
  FROM
  Movie
  JOIN M_Cast
  ON Movie.MID = M_Cast.MID
  WHERE
  CAST(SUBSTR(Movie.year,-4) AS INTEGER) > 1990
)
SELECT
DISTINCT
TRIM(Person.Name) Actor_Names
FROM
query_less_1970
JOIN query_more_1990
ON query_less_1970.PID = query_more_1990.PID
JOIN Person
ON query_less_1970.PID = TRIM(Person.PID)
"""
grader_3(query3)

        Actor_Names
0      Rishi Kapoor
1  Amitabh Bachchan
2            Asrani
3      Zohra Sehgal
4   Parikshat Sahni
5     Rakesh Sharma
6       Sanjay Dutt
7         Ric Young
8             Yusuf
9    Suhasini Mulay
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 406 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [ ]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10))
    return (query_4a.shape == (1462,2))

query_4a ="""
WITH
dir_mov_id_cnt AS
(
  SELECT
  TRIM(M_Director.PID) Director_ID,
  COUNT(M_Director.MID) Movie_count
  FROM
  M_Director
  GROUP BY
  M_Director.PID
)
SELECT
DISTINCT
Director_ID,
Movie_count
FROM
dir_mov_id_cnt
ORDER BY
Movie_count DESC
"""
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

  Director_ID  Movie_count
0   nm0223522           39
1   nm0080315           35
2   nm0698184           30
3   nm0890060           30
4   nm0080333           29
5   nm0611531           27
6   nm0007181           21
7   nm0154113           19
8   nm0759662           19
9   nm0007131           18
True
CPU times: user 9.72 ms, sys: 5 µs, total: 9.73 ms
Wall time: 10.1 ms


In [ ]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 = """
SELECT
DISTINCT
TRIM(Person.Name) Director_Name,
MC.Movie_Count
FROM
(
  SELECT
  PID,
  COUNT(MID) Movie_Count
  FROM
  M_Director
  GROUP BY
  PID
  HAVING
  Movie_Count >= 10
) MC
JOIN Person
ON TRIM(MC.PID) = TRIM(Person.PID)
ORDER BY
MC.Movie_Count DESC
"""
grader_4(query4)

          Director_Name  Movie_Count
0          David Dhawan           39
1          Mahesh Bhatt           35
2          Priyadarshan           30
3       Ram Gopal Varma           30
4          Vikram Bhatt           29
5  Hrishikesh Mukherjee           27
6           Yash Chopra           21
7       Basu Chatterjee           19
8        Shakti Samanta           19
9          Subhash Ghai           18
CPU times: user 443 ms, sys: 5 ms, total: 448 ms
Wall time: 444 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [ ]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10))
    return (query_5aa.shape == (8846,3))

query_5aa ="""
WITH
mid_gender_cnt AS
(
  SELECT
  TRIM(Movie.MID) MID,
  Person.Gender AS Gend,
  COUNT(*) AS Count
  FROM Movie
  JOIN M_Cast
  ON M_Cast.MID = Movie.MID
  JOIN Person
  ON Person.PID = TRIM(M_Cast.PID)
  GROUP BY
  Movie.MID, Person.Gender
)
SELECT mid_gender_cnt.MID, mid_gender_cnt.Gend, mid_gender_cnt.Count
FROM mid_gender_cnt
"""
print(grader_5aa(query_5aa))



def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10))
    return (query_5ab.shape == (3469, 3))

query_5ab ="""
WITH
male_actors_only AS
(
  SELECT
  TRIM(Movie.MID) MID,
  TRIM(Person.Gender) AS Gend,
  COUNT(Person.Gender IN ('Male',NULL)) AS Count
  FROM Movie
  JOIN M_Cast
  ON M_Cast.MID = Movie.MID
  JOIN Person
  ON Person.PID = TRIM(M_Cast.PID)
  WHERE
  Person.Gender='Male'
  GROUP BY
  Movie.MID
)
SELECT male_actors_only.MID, male_actors_only.Gend, male_actors_only.Count
FROM male_actors_only
"""

print(grader_5ab(query_5ab))

# using the above queries, you can write the answer to the given question

         MID    Gend  Count
0  tt0021594    None      1
1  tt0021594  Female      3
2  tt0021594    Male      5
3  tt0026274    None      2
4  tt0026274  Female     11
5  tt0026274    Male      9
6  tt0027256    None      2
7  tt0027256  Female      5
8  tt0027256    Male      8
9  tt0028217  Female      3
True
         MID  Gend  Count
0  tt0021594  Male      5
1  tt0026274  Male      9
2  tt0027256  Male      8
3  tt0028217  Male      7
4  tt0031580  Male     27
5  tt0033616  Male     46
6  tt0036077  Male     11
7  tt0038491  Male      7
8  tt0039654  Male      6
9  tt0040067  Male     10
True
CPU times: user 626 ms, sys: 17 ms, total: 643 ms
Wall time: 639 ms


In [ ]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))

query5a = """
WITH
male_only_movies AS
(
  SELECT
  TRIM(M_Cast.MID)
  FROM
  M_Cast
  INNER JOIN Person
  ON Person.PID = TRIM(M_Cast.PID)
  WHERE
  Person.Gender IN ('Male', NULL)
  GROUP BY
  M_Cast.MID
),
female_only_movies AS
(
  SELECT
  Movie.MID
  FROM Movie
  INNER JOIN M_Cast
  ON TRIM(M_Cast.MID) = Movie.MID
  WHERE TRIM(Movie.MID)
  NOT IN male_only_movies
  AND M_Cast.PID NOTNULL
  GROUP BY
  Movie.MID
)
SELECT
CAST(SUBSTR(Movie.year,-4) AS INTEGER) year,
COUNT(*) AS 'count'
FROM Movie
WHERE
TRIM(Movie.MID) IN female_only_movies
GROUP BY
year
ORDER BY
year
"""
grader_5a(query5a)

   year  count
0  1939      1
1  1999      1
2  2000      1
3  2018      1
CPU times: user 266 ms, sys: 4.99 ms, total: 271 ms
Wall time: 272 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [ ]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = """
WITH
male_only_movies AS
(
  SELECT
  TRIM(M_Cast.MID)
  FROM
  M_Cast
  INNER JOIN Person
  ON Person.PID = TRIM(M_Cast.PID)
  WHERE
  Person.Gender IN ('Male', NULL)
  GROUP BY
  M_Cast.MID
),
female_only_movies AS
(
  SELECT
  Movie.MID
  FROM Movie
  INNER JOIN M_Cast
  ON TRIM(M_Cast.MID) = Movie.MID
  WHERE TRIM(Movie.MID)
  NOT IN male_only_movies
  AND M_Cast.PID NOTNULL
  GROUP BY
  Movie.MID
),
total_movies_yr AS
(
  SELECT
  CAST(SUBSTR(Movie.year,-4) AS INTEGER) year, COUNT(*) AS 'total_movies'
  FROM Movie
  GROUP BY
  CAST(SUBSTR(Movie.year,-4) AS INTEGER)
)
SELECT CAST(SUBSTR(Movie.year,-4) AS INTEGER) year, COUNT(CAST(SUBSTR(Movie.year,-4) AS INTEGER)) * 1.0 /total_movies_yr.total_movies AS Percentage_Female_Only_Movie, total_movies_yr.total_movies
FROM Movie
INNER JOIN female_only_movies
ON female_only_movies.MID = Movie.MID
INNER JOIN total_movies_yr
ON CAST(SUBSTR(Movie.year,-4) AS INTEGER) = total_movies_yr.year
GROUP BY
Movie.year
ORDER BY
Movie.year
"""
grader_5b(query5b)

   year  Percentage_Female_Only_Movie  total_movies
0  1939                      0.500000             2
1  1999                      0.015152            66
2  2000                      0.015625            64
3  2018                      0.009615           104
CPU times: user 251 ms, sys: 1.99 ms, total: 253 ms
Wall time: 252 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [ ]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = """
WITH
large_cast_films AS
(
  SELECT
  COUNT(*) AS COUNT_OF_CAST,
  MID
  FROM M_Cast
  GROUP BY
  MID
)
SELECT
Movie.title, large_cast_films.COUNT_OF_CAST AS count
FROM large_cast_films
JOIN Movie
ON Movie.MID = large_cast_films.MID
ORDER BY
count DESC
"""
grader_6(query6)

                        title  count
0               Ocean's Eight    238
1                    Apaharan    233
2                        Gold    215
3             My Name Is Khan    213
4  Captain America: Civil War    191
5                    Geostorm    170
6                     Striker    165
7                        2012    154
8                      Pixels    144
9       Yamla Pagla Deewana 2    140
CPU times: user 53.2 ms, sys: 1.99 ms, total: 55.2 ms
Wall time: 55.7 ms


### Q7 --- A decade is a sequence of 10 consecutive years.
### For example, say in your database you have movie information starting from 1931.
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on.
### Find the decade D with the largest number of films and the total number of films in D

In [ ]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

query7a = """
WITH
number_of_movies_each_year AS
(
  SELECT
  CAST(SUBSTR(Movie.year,-4) AS INTEGER) Movie_Year,
  COUNT(DISTINCT MID) Total_Movies
  FROM
  Movie
  GROUP BY
  CAST(SUBSTR(Movie.year,-4) AS INTEGER)
)
SELECT
Movie_Year,
Total_Movies
FROM
number_of_movies_each_year
"""
grader_7a(query7a)

# using the above query, you can write the answer to the given question

   Movie_Year  Total_Movies
0        1931             1
1        1936             3
2        1939             2
3        1941             1
4        1943             1
5        1946             2
6        1947             2
7        1948             3
8        1949             3
9        1950             2
CPU times: user 6.09 ms, sys: 673 µs, total: 6.76 ms
Wall time: 8.38 ms


In [ ]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

query7b = """
WITH
number_of_movies_each_year AS
(
  SELECT
  CAST(SUBSTR(Movie.year,-4) AS INTEGER) Movie_Year,
  COUNT(DISTINCT MID) Total_Movies
  FROM
  Movie
  GROUP BY
  Movie_Year
)
SELECT *
FROM
number_of_movies_each_year AS table1,
number_of_movies_each_year AS table2
WHERE
table2.Movie_Year <= table1.Movie_Year+9 AND table2.Movie_Year >= table1.Movie_Year
GROUP BY table1.Movie_Year, table2.Movie_Year
ORDER BY table1.Movie_Year, table2.Movie_Year
"""
grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

   Movie_Year  Total_Movies  Movie_Year  Total_Movies
0        1931             1        1931             1
1        1931             1        1936             3
2        1931             1        1939             2
3        1936             3        1936             3
4        1936             3        1939             2
5        1936             3        1941             1
6        1936             3        1943             1
7        1939             2        1939             2
8        1939             2        1941             1
9        1939             2        1943             1
CPU times: user 12.1 ms, sys: 1 µs, total: 12.1 ms
Wall time: 11.4 ms


In [ ]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = """
WITH
distinct_years AS
(
  SELECT
  DISTINCT
  CAST(SUBSTR(Movie.year,-4) AS INTEGER) year,
  CAST(SUBSTR(Movie.year,-4) AS INTEGER) decade_start,
  CAST(SUBSTR(Movie.year,-4) AS INTEGER)+9 decade_end,
  SUBSTR(Movie.year,-4) decade
  FROM
  Movie
),
number_of_movies_each_year AS
(
  SELECT
  CAST(SUBSTR(Movie.year,-4) AS INTEGER) Movie_Year,
  COUNT(DISTINCT MID) Total_Movies
  FROM
  Movie
  GROUP BY
  CAST(SUBSTR(Movie.year,-4) AS INTEGER)
),
number_of_movies_in_a_decade AS
(
  SELECT
  SUM(Total_Movies) Decade_Movie_Count,
  distinct_years.decade
  FROM number_of_movies_each_year,
  distinct_years
  WHERE
  number_of_movies_each_year.Movie_Year BETWEEN distinct_years.decade_start AND distinct_years.decade_end
  GROUP BY
  distinct_years.decade
)
SELECT
Decade_Movie_Count,
decade AS Decade
FROM
number_of_movies_in_a_decade
WHERE
Decade_Movie_Count = (
  SELECT
  MAX(Decade_Movie_Count)
  FROM
  number_of_movies_in_a_decade
)
"""
grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

   Decade_Movie_Count Decade
0                1203   2008
CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 9.64 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [ ]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = """
WITH actor_director_movies AS
(
  SELECT
  DISTINCT
  TRIM(M_Cast.PID) actor,
  TRIM(M_Director.PID) director,
  COUNT(DISTINCT TRIM(M_Director.MID)) AS movies
  FROM
  M_Cast,
  M_Director
  WHERE
  TRIM(M_Cast.MID) = TRIM(M_Director.MID)
  GROUP BY
  actor,
  director
)
SELECT
DISTINCT
actor,
director,
movies
FROM
actor_director_movies
"""
grader_8a(query8a)

# using the above query, you can write the answer to the given question

       actor   director  movies
0  nm0000002  nm0496746       1
1  nm0000027  nm0000180       1
2  nm0000039  nm0896533       1
3  nm0000042  nm0896533       1
4  nm0000047  nm0004292       1
5  nm0000073  nm0485943       1
6  nm0000076  nm0000229       1
7  nm0000092  nm0178997       1
8  nm0000093  nm0000269       1
9  nm0000096  nm0113819       1
CPU times: user 1min 3s, sys: 93.1 ms, total: 1min 3s
Wall time: 1min 4s


In [ ]:
 %%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = """
SELECT
TRIM(Person.Name) AS Actor_Name,
movie_count
FROM
(
  SELECT
  actor_pid,
  director_pid,
  movie_count
  FROM
  (
    SELECT
    TRIM(M_Cast.PID) AS actor_pid,
    TRIM(M_Director.PID) AS director_pid,
    COUNT(*) AS movie_count
    FROM M_Cast
    JOIN M_Director
    ON TRIM(M_Cast.MID) = TRIM(M_Director.MID)
    GROUP BY
    actor_pid,
    director_pid
  )
  WHERE
  (actor_pid,movie_count)
  IN
  (
    SELECT
    actor_pid,
    MAX(movie_count)
    FROM
    (
      SELECT
      TRIM(M_Cast.PID) AS actor_pid,
      TRIM(M_Director.PID) AS director_pid,
      COUNT(*) AS movie_count
      FROM M_Cast
      JOIN M_Director
      ON TRIM(M_Cast.MID) = TRIM(M_Director.MID)
      GROUP BY
      actor_pid,
      director_pid
    )
    GROUP BY
    actor_pid
  )
  AND director_pid = (SELECT TRIM(Person.PID) FROM Person WHERE TRIM(Person.Name) LIKE '%Yash Chopra%')
)
JOIN Person
ON TRIM(Person.PID) = actor_pid
ORDER BY
movie_count DESC
"""
grader_8(query8)

         Actor_Name  movie_count
0       Jagdish Raj           11
1  Manmohan Krishna           10
2          Iftekhar            9
3     Shashi Kapoor            7
4    Waheeda Rehman            5
5     Rakhee Gulzar            5
6    Achala Sachdev            4
7       Neetu Singh            4
8          Ravikant            4
9   Parikshat Sahni            3
(245, 2)
CPU times: user 2min 3s, sys: 159 ms, total: 2min 3s
Wall time: 2min 4s


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [ ]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """
WITH
S0 AS
(
  SELECT
  TRIM(Person.PID) PID
  FROM
  Person
  WHERE
  TRIM(Person.Name) LIKE '%Shah Rukh Khan%'
),
S1_movies AS
(
  SELECT
  DISTINCT
  TRIM(M_Cast.MID) MID,
  S0.PID
  FROM
  M_Cast,
  S0
  WHERE
  TRIM(M_Cast.PID) = S0.PID
),
S1 AS
(
  SELECT
  DISTINCT
  TRIM(M_Cast.PID) PID
  FROM
  M_Cast,
  S1_movies
  WHERE
  TRIM(M_Cast.MID) = S1_movies.MID AND
  TRIM(M_Cast.PID) != S1_movies.PID
)
SELECT
DISTINCT
PID AS S1_PID
FROM S1
"""
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

      S1_PID
0  nm0004418
1  nm1995953
2  nm2778261
3  nm0631373
4  nm0241935
5  nm0792116
6  nm1300111
7  nm0196375
8  nm1464837
9  nm2868019
(2382, 1)
CPU times: user 127 ms, sys: 3.98 ms, total: 131 ms
Wall time: 147 ms


In [ ]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """
SELECT
DISTINCT
TRIM(M_Cast.PID) S2_PID
FROM
M_Cast
WHERE
TRIM(M_Cast.MID)
IN
(
  SELECT
  DISTINCT
  TRIM(M_Cast.MID)
  FROM
  M_Cast
  WHERE
  TRIM(M_Cast.PID)
  IN
  (
    SELECT
    DISTINCT
    TRIM(M_Cast.PID) S1
    FROM
    M_Cast
    WHERE
    M_Cast.MID
    IN
    (
      SELECT
      M_Cast.MID
      FROM
      M_Cast
      WHERE
      TRIM(M_Cast.PID) = (SELECT TRIM(PID) FROM Person WHERE TRIM(Name) LIKE '%Shah Rukh Khan%')
    )
    AND S1 != (SELECT TRIM(PID) FROM Person WHERE TRIM(Name) LIKE '%Shah Rukh Khan%')
  )
)
AND S2_PID NOT IN
(
  SELECT
  DISTINCT
  TRIM(M_Cast.PID) actor_id
  FROM
  M_Cast
  WHERE
  M_Cast.MID
  IN
  (
    SELECT
    M_Cast.MID
    FROM
    M_Cast
    WHERE
    TRIM(M_Cast.PID) = (SELECT TRIM(PID) FROM Person WHERE TRIM(Name) LIKE '%Shah Rukh Khan%')
  )
  AND actor_id != (SELECT TRIM(PID) FROM Person WHERE TRIM(Name) LIKE '%Shah Rukh Khan%')
)
AND S2_PID != (SELECT TRIM(PID) FROM Person WHERE TRIM(Name) LIKE '%Shah Rukh Khan%')
"""

grader_9(query9)

      S2_PID
0  nm2539953
1  nm0922035
2  nm0324658
3  nm0943079
4  nm0000218
5  nm0001394
6  nm0929654
7  nm3116102
8  nm3248891
9  nm2418809
(25698, 1)
CPU times: user 389 ms, sys: 15 ms, total: 404 ms
Wall time: 424 ms
